# SecDev Sampling Tutorial

## Testing Imports

In [ ]:
// Importing the local CryptoGuard Jar
List <String> addedJars = %jars *.jar
//*/

// Maven Imports
%maven junit:junit:4.12
%maven org.apache.commons:commons-io:1.3.2
%maven commons-io:commons-io:2.7
%maven com.binarytweed:quarantining-test-runner:0.0.3
%maven org.knowm.xchart:xchart:3.5.2

// Gradle imports
import static org.junit.Assert.assertFalse;
import static org.junit.Assert.assertTrue;
import static org.junit.Assert.assertEquals;
import static org.junit.Assert.assertNull;
import static org.junit.Assert.assertNotNull;
import static util.Utils.makeArg;
import frontEnd.Interface.outputRouting.ExceptionHandler;
import frontEnd.Interface.ArgumentsCheck;
import frontEnd.MessagingSystem.routing.Listing;
import frontEnd.MessagingSystem.routing.EnvironmentInformation;
import frontEnd.MessagingSystem.routing.structure.Default.Report;
import frontEnd.MessagingSystem.routing.structure.Scarf.AnalyzerReport;
import frontEnd.MessagingSystem.routing.structure.Scarf.BugInstance;
import frontEnd.Interface.EntryPoint;
import rule.engine.EngineType;
import frontEnd.argsIdentifier;
import java.io.File;
import java.nio.charset.StandardCharsets;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import org.junit.After;
import org.junit.Before;
import org.junit.Test;
import org.junit.runner.RunWith;
import soot.G;
import util.Utils;
import org.apache.commons.io.FileUtils;
import java.util.Scanner;
import org.apache.commons.io.filefilter.WildcardFileFilter;
import com.binarytweed.test.Quarantine;
import com.binarytweed.test.QuarantiningRunner;
import java.io.File;

//To Add the slides and diagrams
//https://github.com/knowm/XChart
import org.knowm.xchart.*;

## What version of CryptoGuard is this notebook using?

In [ ]:
System.out.println("Cryptoguard Version: " + Utils.projectVersion);

In [ ]:
%%writefile import_sha.java

import java.math.BigInteger;
import java.security.MessageDigest;
import java.security.NoSuchAlgorithmException;
import java.nio.charset.StandardCharsets;

public class import_sha {
	public static String encrypt(String input) {
		try {
			MessageDigest md = MessageDigest.getInstance("SHA-512");
		
			byte[] messageDigest = md.digest(input.getBytes());

			return new String(messageDigest,StandardCharsets.UTF_8);
		} catch(NoSuchAlgorithmException e) {
			throw new RuntimeException(e);
		}
	}

	public static void main(String[] args) {
		String string = "Hello World";

		System.out.println("Hashing the string: " + string);
		System.out.println("The hashed value is:" + encrypt(string));
	}
}

## Loading the Custom Java And APK Versions From [SDKMan Downloads](https://sdkman.io/)

In [ ]:
//Defining the base username and the paths to SDK Download JDKs
String cur_user = System.getProperty("user.name");
String java_home = "/home/" + cur_user + "/.sdkman/candidates/java/8*";
String java7_home = "/home/" + cur_user + "/.sdkman/candidates/java/7*";
String android_home = "/home/" + cur_user + "/.sdkman/candidates/android/current";

//Retrieving a specific path from a wild card based list
/*
 * This is used since SDKMan Vendors will inconsistently upgrade minor versions of supported JDKs.
 * When installing a JDK through SDKMan we need to specific a specific JDK version (specifying minor version).
 * We "dynamically" download a JDK through SDKMan but have to look for the wild-card type expression to add it onto the path.
**/
public static String getFileFromWildCard(String path) {
	String[] split = path.split("/");
	StringBuilder build = new StringBuilder();
	for (int itr = 0;itr < split.length - 1;itr ++)
		build.append(split[itr]).append("/");

	String[] files = new File(build.toString()).list(new WildcardFileFilter(split[split.length-1]));

	if (files.length > 0)
		return build.toString() + files[files.length - 1];
	else
		return null;
}

java_home = getFileFromWildCard(java_home);
java7_home = getFileFromWildCard(java7_home);

## Test Setup

In [ ]:
// region String Statics
public static final Boolean isLinux = !System.getProperty("os.name").contains("Windows");

//Defining the different paths we will use
public static final String basePath = System.getProperty("user.dir");
public static final String testRec = Utils.osPathJoin(basePath, "samples");
public static final String testPath = Utils.osPathJoin(basePath, "tmp");

//Clearing the tmp directory
FileUtils.deleteDirectory(new File(testPath));
new File(testPath).mkdirs();

//Setting and creating the timing methods
public long startTime;
public long endTime;

public static void start(){
	startTime = System.currentTimeMillis();
}

public static void end(){
	endTime = System.currentTimeMillis();
	System.out.println("Total Execution time: " + (endTime - startTime) + " ms");
}


//Resetting the file (deleting and re-creating the file).
public static void create(String path) {
	File newFile = new File(path);
	try{
		newFile.delete();
		newFile.createNewFile();
	} catch(Exception e) {
	}
}

//Printing the contents of the file
public static void print(String file) {
	try {
		Scanner reader = new Scanner(new File(file));

		while (reader.hasNextLine()) 
			System.out.println(reader.nextLine());
	}
	catch (FileNotFoundException e) {
	}
}

## APK Setup

In [ ]:
public static final String tempFileOutApk = Utils.osPathJoin(testPath, "app-debug.txt");
public static final String tempFileOutApk_XML = Utils.osPathJoin(testPath, "app-debug.xml");
public static final String tempFileOutApk_JSON = Utils.osPathJoin(testPath, "app-debug.json");
public static final String tempFileOutApk_CSV = Utils.osPathJoin(testPath, "app-debug.csv");

create(tempFileOutApk);
create(tempFileOutApk_XML);
create(tempFileOutApk_JSON);
create(tempFileOutApk_CSV);

public static final String pathToAPK = Utils.osPathJoin(testRec, "app-debug.apk");

## APK Tests

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableApk_Legacy {
	public static void apply() {
	G.reset();
	String fileOut = tempFileOutApk;

	new File(fileOut).delete();
	if (isLinux) {
		String args =
			makeArg(argsIdentifier.FORMAT, EngineType.APK)
				+ makeArg(argsIdentifier.SOURCE, pathToAPK)
				+ makeArg(argsIdentifier.FORMATOUT, Listing.Legacy)
				+ makeArg(argsIdentifier.ANDROID, android_home)
				+ makeArg(argsIdentifier.JAVA, java_home)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.OUT, fileOut);

		try {
			System.out.println("CMD Line: " + args);
			EntryPoint.main(args.split(" "));
			print(fileOut);
		} catch (Exception e) {
			e.printStackTrace();
			assertNull(e);
		}
	  }
	}
}

In [ ]:
start();
main_TestableApk_Legacy.apply();
end();

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableApk_XML {
	public static void apply() {
		G.reset();
		String fileOut = tempFileOutApk_XML;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.APK)
					+ makeArg(argsIdentifier.SOURCE, pathToAPK)
					+ makeArg(argsIdentifier.FORMATOUT, Listing.ScarfXML)
					+ makeArg(argsIdentifier.ANDROID, android_home)
					+ makeArg(argsIdentifier.JAVA, java_home)
					+ makeArg(argsIdentifier.NOEXIT)
					+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableApk_XML.apply();
end();

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableApk_JSON {
	public static void apply() {
		G.reset();
		String fileOut = tempFileOutApk_JSON;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.APK)
					+ makeArg(argsIdentifier.SOURCE, pathToAPK)
					+ makeArg(argsIdentifier.ANDROID, android_home)
					+ makeArg(argsIdentifier.JAVA, java_home)
					+ makeArg(argsIdentifier.NOEXIT)
					+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableApk_JSON.apply();
end();

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableApk_CSV {
	public static void apply() {
		G.reset();
		String fileOut = tempFileOutApk_CSV;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.APK)
					+ makeArg(argsIdentifier.SOURCE, pathToAPK)
					+ makeArg(argsIdentifier.FORMATOUT, Listing.CSVDefault)
					+ makeArg(argsIdentifier.ANDROID, android_home)
					+ makeArg(argsIdentifier.JAVA, java_home)
					+ makeArg(argsIdentifier.NOEXIT)
					+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableApk_CSV.apply();
end();

## Java Class File Setup

In [ ]:
public static final String verySimple_Klass_xml_1 = Utils.osPathJoin(testPath, "verySimple_klass_1.xml");
public static final String verySimple_Klass_yaml_1 = Utils.osPathJoin(testPath, "verySimple_klass_2.yaml");

create(verySimple_Klass_xml_1);
create(verySimple_Klass_yaml_1);

public static final String verySimple_Path = Utils.osPathJoin(testRec, "VerySimple");
public static final String verySimple_Klass = Utils.osPathJoin(verySimple_Path, "very.class");

## Java Class Tests

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableFile_VerySimple {
	public static void apply() {
		G.reset();
		soot.G.v().reset();

		String source = verySimple_Klass;
		String fileOut = verySimple_Klass_xml_1;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.CLASSFILES)
				+ makeArg(argsIdentifier.FORMATOUT, Listing.ScarfXML)
				+ makeArg(argsIdentifier.SOURCE, source)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
				+ makeArg(argsIdentifier.VERYVERBOSE)
				+ makeArg(argsIdentifier.STREAM)
				+ makeArg(argsIdentifier.PRETTY)
				+ makeArg(argsIdentifier.JAVA, java_home)
				+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableFile_VerySimple.apply();
end();

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableFile_VerySimple_YAML {
	public static void apply() {
		G.reset();
		soot.G.v().reset();

		String source = verySimple_Klass;
		String fileOut = verySimple_Klass_yaml_1;
		Listing type = Listing.YAMLGeneric;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.CLASSFILES)
				+ makeArg(argsIdentifier.FORMATOUT, type)
				+ makeArg(argsIdentifier.SOURCE, source)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
				+ makeArg(argsIdentifier.VERYVERBOSE)
				+ makeArg(argsIdentifier.PRETTY)
				+ makeArg(argsIdentifier.JAVA, java_home)
				+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableFile_VerySimple_YAML.apply();
end();

## Jar Setup

In [ ]:
public static final String tempJarFile_xml = Utils.osPathJoin(testPath, "tempJarFile.xml");
public static final String tempJarFile_json = Utils.osPathJoin(testPath, "tempJarFile.json");

create(tempJarFile_xml);
create(tempJarFile_json);

public static final String jarOne = Utils.osPathJoin(testRec, "testable-jar", "build", "libs", "testable-jar.jar");
public static final String verySimple_Jar = Utils.osPathJoin(verySimple_Path, "very.jar");
public static final String srcOneGrvDep = Utils.osPathJoin(testRec, "testable-jar", "build", "dependencies");

## JAR Tests

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_VerySimple_Scarf {
	public static void apply() {
		G.reset();
		String fileOut = tempJarFile_xml;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.JAR)
				+ makeArg(argsIdentifier.SOURCE, verySimple_Jar)
				+ makeArg(argsIdentifier.FORMATOUT, Listing.ScarfXML)
				+ makeArg(argsIdentifier.OUT, fileOut)
				+ makeArg(argsIdentifier.STREAM)
				+ makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
				+ makeArg(argsIdentifier.JAVA, java_home)
				+ makeArg(argsIdentifier.TIMEMEASURE)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.PRETTY);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_VerySimple_Scarf.apply();
end();

In [ ]:
@RunWith(QuarantiningRunner.class)
@Quarantine({"com.binarytweed", "*"})
public class main_TestableJar {
	public static void apply() {
		G.reset();
		String fileOut = tempJarFile_json;

		new File(fileOut).delete();
		if (isLinux) {
			String args =
				makeArg(argsIdentifier.FORMAT, EngineType.JAR)
				+ makeArg(argsIdentifier.FORMATOUT, Listing.Default)
				+ makeArg(argsIdentifier.SOURCE, jarOne)
				+ makeArg(argsIdentifier.ANDROID, "/InvalidPath/")
				+ makeArg(argsIdentifier.JAVA, java_home)
				+ makeArg(argsIdentifier.DEPENDENCY, srcOneGrvDep)
				+ makeArg(argsIdentifier.NOEXIT)
				+ makeArg(argsIdentifier.OUT, fileOut);

			try {
				System.out.println("CMD Line: " + args);
				EntryPoint.main(args.split(" "));
				print(fileOut);
			} catch (Exception e) {
				e.printStackTrace();
				assertNull(e);
			}
		}
	}
}

In [ ]:
start();
main_TestableJar.apply();
end();